# 用pandas和pickle加载数据并保存结果
在本例中，我们将展示如何加载数据以便使用PyDESeq2执行DEA分析，以及如何使用pandas和pickle解决其结果。

关于分析本身的更多细节，我们参考入门示例

In [1]:
import os
import pickle as pkl
import sys
sys.path.insert(0, 
                "/slurm/home/admin/nlp/DL/97-bioinformatics/bio_package/pydeseq2")

import pandas as pd

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

# Replace this with the path to directory where you would like results to be saved
OUTPUT_PATH = "/slurm/home/admin/nlp/DL/results/synthetic_example"
os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create path if it doesn't exist

## 使用pandas加载数据
要执行差分表达式分析(DEA)， PyDESeq2需要两种类型的输入

- 样本形状数x基因数的计数矩阵，包含读取计数(非负整数)
- 样本数x变量数的元数据(或注释，或列数据)，包含将用于在队列中拆分数据的样本注释。

两者都应该作为pandas数据框架提供。

这里，我们将展示如何使用`pandas.read_csv()`将与计数和注释相对应的CSV加载为pandas数据框。

我们假设`DATA PATH`是一个包含`test_counts.csv`和`test_metadata.csv`文件的目录

In [ ]:
# Replace this with the path to your dataset
DATA_PATH = "https://raw.githubusercontent.com/owkin/PyDESeq2/main/datasets/synthetic/"

In [ ]:
counts_df = pd.read_csv(os.path.join(DATA_PATH, "test_counts.csv"), index_col=0)
print(counts_df)

注意，计数数据采用基因数x样本数格式，而样本数x基因数是必需的。为了解决这个问题，我们调换计数数据帧。

In [ ]:
counts_df = counts_df.T
print(counts_df)

In [ ]:
metadata = pd.read_csv(os.path.join(DATA_PATH, "test_metadata.csv"), index_col=0)
print(metadata)

在本例中，元数据包含两列，`condition`和`group`，表示两种类型的双层注释。这里，我们只使用`condition`因子。

## 数据过滤

在继续使用DEA之前，我们首先预处理数据，如[入门示例](https://pydeseq2.readthedocs.io/en/latest/auto_examples/plot_minimal_pydeseq2_pipeline.html)中所示。

In [ ]:
samples_to_keep = ~metadata.condition.isna()
counts_df = counts_df.loc[samples_to_keep]
metadata = metadata.loc[samples_to_keep]

## 单因素分析

与入门示例一样，我们忽略了group变量，并使用condition列作为设计因素。

### 读取使用DESEQDATASET类建模

我们首先从刚刚加载的计数和元数据数据中创建一个`deseqdataset`对象。

In [ ]:
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=counts_df,
    metadata=metadata,
    design_factors="condition",
    refit_cooks=True,
    inference=inference,
)

一旦初始化了`DeseqDataSet`，我们就可以运行`deseq2()`方法来拟合分散和lfc。

In [ ]:
dds.deseq2()

DeseqDataSet类继承了AnnData类。

In [ ]:
print(dds)

因此，可以使用`pickle.dump`保存它

In [ ]:
with open(os.path.join(OUTPUT_PATH, "dds.pkl"), "rb") as f:
    dds2 = pkl.load(f)

print(dds2)

参数根据AnnData数据结构存储，并带有基于键的数据字段。特别是
- `X`存储计数数据
- `Obs`存储设计因素
- `Obsm`存储样本级数据，例如`design_matrix` `size_factors`
- `arm`存储基因级别的数据，例如`dispersions` and `LFC`

作为一个例子，下面是我们如何访问色散和lfc(在自然对数尺度下)

In [ ]:
print(dds.varm["dispersions"])

In [ ]:
print(dds.varm["LFC"])

## 使用DeseqStats类进行统计分析

现在已经拟合了离散度和lfc，我们可以进行统计检验来计算p值和调整p值用于差分表达。这是DeseqStats类的角色。

In [ ]:
stat_res = DeseqStats(dds, inference=inference)

PyDESeq2使用Wald检验计算p值。这可以使用summary()方法来完成，该方法运行整个统计分析，包括烹饪过滤和多个测试调整。

In [ ]:
stat_res.summary()

然后将结果存储在`results_df`属性中(`stat_res.results_df`)。与DeseqDataSet一样，整个DeseqStats对象可以使用`pickle`保存。但是，将结果作为CSV格式通常更方便。因此，我们可以使用`pandas.datafframe.to_csv()`将`stats_res.results`导出为CSV。

In [ ]:
stat_res.results_df.to_csv(os.path.join(OUTPUT_PATH, "results.csv"))